## Описание датасета
Датасет состоит из двух файлов: `TrainData3.csv` и `TestData3.csv`, содержащих тренировочные и тестовые данные соответственно. Каждый файл включает в себя анонимизированные характеристики клиентов некоторой компании и целевую переменную, указывающую, покинет ли клиент компанию.
- TrainData3.csv: Тренировочный набор данных, содержащий 1000 записей.
- TestData3.csv: Тестовый набор данных, содержащий 300 записей.

Каждый файл включает 15 столбцов, из которых 14 - это анонимизированные признаки клиентов, а 15-й столбец - целевая переменная.
1. feature_0  до feature_13: Анонимизированные числовые признаки клиентов. Значения этих признаков варьируются от 0 до 100.
2. target: Целевая переменная (0 или 1). Показывает, покинет ли клиент компанию:
- `1`: Клиент покинет компанию
- `0`: Клиент останется в компании

## Задача оттока клиентов
### Часть 1: Работа с пропусками
#### Задание 1
Проверьте, есть ли в тренировочных и тестовых данных пропуски? Укажите количество столбцов тренировочной выборки, имеющих пропуски.

#### Задание 2
1. В столбце с наибольшим количеством пропусков заполните пропуски средним значением по столбцу. В ответ запишите значение вычисленного среднего. Ответ округлите до десятых.
2. Найдите строки в тренировочных данных, где пропуски стоят в столбце с наименьшим количеством пропусков. Удалите эти строки. Сколько строк вы удалили?

### Часть 2: Предобработка данных
#### Задание 3
Выполните следующие пункты только по таблице train.
1. Сколько столбцов в таблице (не считая target) содержат меньше 5 различных значений?
2. Вычислите долю ушедших из компании клиентов, для которых значение признака 2 больше среднего значения по столбцу, а значение признака 13 меньше медианы по столбцу. Ответ округлите до сотых.

### Часть 3: Обучение модели
#### Задание 4
1. Разбейте тренировочные данные на целевой вектор y, содержащий значения из столбца target, и матрицу объект-признак X, содержащую остальные признаки. Обучите на этих данных логистическую регрессию из sklearn (LogisticRegression) с параметрами по умолчанию. Выведите среднее значение метрики f1-score алгоритма на кросс-валидации с тремя фолдами. Ответ округлите до сотых.
- При объявлении модели фиксируйте random_state = 42.
- Комментарий: параметры по умолчанию можете оставить дефолтными

#### Задание 5
1. Подберите значение константы регуляризации C в логистической регрессии, перебирая гиперпараметр от 0.001 до 100 включительно, проходя по степеням 10. Для выбора C примените перебор по сетке по тренировочной выборке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - f1-score. Остальные параметры оставьте по умолчанию. В ответ запишите наилучшее среди искомых значение C.
- При объявлении модели фиксируйте random_state = 42.
- Комментарий: параметры по умолчанию можете оставить дефолтными

2. Добавьте в тренировочные и тестовые данные новый признак 'NEW', равный произведению признаков '7' и '11'. На тренировочных данных с новым признаком заново с помощью GridSearchCV (с тремя фолдами и метрикой качества - f1-score) подберите оптимальное значение C (перебирайте те же значения C, что и в предыдущих заданиях), в ответ напишите наилучшее качество алгоритма (по метрике f1-score), ответ округлите до сотых.
- При объявлении модели фиксируйте random_state = 42.

3. Теперь вы можете использовать любую модель машинного обучения для решения задачи. Также можете делать любую другую обработку признаков. Ваша задача - получить наилучшее качество по метрике F1-Score на тестовых данных.

Лучший результат будет можно будет добавить в викторину в Телеграмм, победитель получит +2 балла к оценке


## Подключение библиотек и датасетов

In [1]:
import os
import csv
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import f1_score, make_scorer

train_df = pd.read_csv(os.path.abspath("TrainData_new.csv"))
test_df = pd.read_csv(os.path.abspath("TestData_new.csv"))

train_df.head()
# test_df.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,cat_feature_1,cat_feature_2,target
0,2.740701,8.506838,47.724081,29.151119,NaN,14.293323,2.255775,51.659614,30.097626,69.505159,1.710745,88.430339,21.172353,16.403757,B,individuals,0
1,1.330506,1.641074,19.835795,25.552527,34.141593,26.684477,23.762340,8.319554,14.627584,35.962469,37.903265,71.587453,5.078505,39.725969,A,individuals,0
2,4.334999,0.415631,39.610327,8.303514,5.141777,86.943615,37.501950,48.213628,15.251909,9.587599,56.865540,40.130363,3.103922,38.253725,C,legal entities,0
3,0.251638,8.136452,16.871775,32.260907,24.638119,47.652316,21.232416,11.024905,48.546775,76.087864,78.080292,631.536797,15.206993,71.217221,C,legal entities,0
4,0.647545,1.753623,2.948700,15.841658,NaN,24.862920,32.185420,21.277126,14.066296,53.271626,11.711993,73.139324,1.896122,76.239624,A,individuals,0


## Часть 1: Работа с пропусками
### Задание 1
Проверьте, есть ли в тренировочных и тестовых данных пропуски? Укажите количество столбцов тренировочной выборки, имеющих пропуски.

In [2]:
# Проверка наличия пропусков в тренировочных и тестовых данных
print(f"Пропуски в тренировочных данных: {train_df.isna().sum().sum()}")
print(f"Пропуски в тестовых данных: {test_df.isna().sum().sum()}")

# Количество столбцов тренировочной выборки, имеющих пропуски
print(f"Количество столбцов тренировочной выборки с пропусками: {train_df.isnull().sum()[train_df.isnull().sum() > 0].shape[0]}")

Пропуски в тренировочных данных: 1172
Пропуски в тестовых данных: 107
Количество столбцов тренировочной выборки с пропусками: 14


### Задание 2
1. В столбце с наибольшим количеством пропусков заполните пропуски средним значением по столбцу. В ответ запишите значение вычисленного среднего. Ответ округлите до десятых.
2. Найдите строки в тренировочных данных, где пропуски стоят в столбце с наименьшим количеством пропусков. Удалите эти строки. Сколько строк вы удалили?

In [3]:
# a) Заполнение пропусков средним значением в столбце с наибольшим количеством пропусков
col_with_most_nans = train_df.isnull().sum().sort_values(ascending=False).index[0]
mean_value = train_df[col_with_most_nans].mean()
train_df[col_with_most_nans].fillna(mean_value, inplace=True)
print(f"Среднее значение для заполнения пропусков: {mean_value:.1f}")

# b) Удаление строк с пропусками в столбце с наименьшим количеством пропусков
col_with_least_nans = train_df.isnull().sum()[train_df.isnull().sum()>0].sort_values(ascending=True).index[0]
rows_to_drop = train_df[train_df[col_with_least_nans].isnull()].shape[0]
train_df = train_df.dropna(subset=[col_with_least_nans])
print(f"Количество удаленных строк: {rows_to_drop}")

Среднее значение для заполнения пропусков: 70.6
Количество удаленных строк: 69


## Часть 2: Предобработка данных
### Задание 3
Выполните следующие пункты только по таблице train.
1. Сколько столбцов в таблице (не считая target) содержат меньше 5 различных значений?
2. Вычислите долю ушедших из компании клиентов, для которых значение признака 2 больше среднего значения по столбцу, а значение признака 13 меньше медианы по столбцу. Ответ округлите до сотых.

In [4]:
# a) Количество столбцов с меньше чем 5 уникальными значениями
num_cols_with_few_unique = sum(train_df.iloc[:, :-1].nunique() < 5)
print(f"Количество столбцов с меньше 5 уникальными значениями: {num_cols_with_few_unique}")

# b) Доля ушедших клиентов, для которых признак 2 больше среднего, а признак 13 меньше медианы
mean_feature2 = train_df['feature_2'].mean()
median_feature13 = train_df['feature_13'].median()
condition = (train_df['target'] == 1) & (train_df['feature_2'] > mean_feature2) & (train_df['feature_13'] < median_feature13)
proportion = train_df[condition].shape[0] / train_df[train_df['target'] == 1].shape[0]
print(f"Доля ушедших клиентов: {proportion:.2f}")

Количество столбцов с меньше 5 уникальными значениями: 2
Доля ушедших клиентов: 0.02


## Часть 3: Обучение модели
### Задание 4
1. Разбейте тренировочные данные на целевой вектор y, содержащий значения из столбца target, и матрицу объект-признак X, содержащую остальные признаки. Обучите на этих данных логистическую регрессию из sklearn (LogisticRegression) с параметрами по умолчанию. Выведите среднее значение метрики f1-score алгоритма на кросс-валидации с тремя фолдами. Ответ округлите до сотых.
- При объявлении модели фиксируйте random_state = 42.
- Комментарий: параметры по умолчанию можете оставить дефолтными

In [6]:
# Разделение на входные данные (X) и целевую переменную (y)
X = train_df.drop('target', axis=1)
y = train_df['target']

# Создание и обучение модели логистической регрессии
model = LogisticRegression(random_state=42)
model.fit(X, y)

# Оценка модели на тестовом наборе
accuracy = accuracy_score(y_test, model.predict(X_test))
print(f'Точность модели: {accuracy:.2f}')

# Использование cross_val_score с модификацией
f1_scores = cross_val_score(model, X, y, cv=3, scoring='f1')
print(f"Среднее значение f1-score: {f1_scores.mean():.2f}")

# заполнить пропуски медианами
# закодировать категории (0 1 и заполнить)

ValueError: could not convert string to float: 'B'

In [ ]:
# a) Обучение логистической регрессии с кросс-валидацией
X = train_df.drop('target', axis=1)
y = train_df['target']

model = LogisticRegression(random_state=42)

# Использование cross_val_score с модификацией
f1_scores = cross_val_score(model, X, y, cv=3, scoring='f1')
print(f"Среднее значение f1-score: {f1_scores.mean():.2f}")

### Задание 5
1. Подберите значение константы регуляризации C в логистической регрессии, перебирая гиперпараметр от 0.001 до 100 включительно, проходя по степеням 10. Для выбора C примените перебор по сетке по тренировочной выборке (GridSearchCV из библиотеки sklearn.model_selection) с тремя фолдами и метрикой качества - f1-score. Остальные параметры оставьте по умолчанию. В ответ запишите наилучшее среди искомых значение C.
- При объявлении модели фиксируйте random_state = 42.
- Комментарий: параметры по умолчанию можете оставить дефолтными

2. Добавьте в тренировочные и тестовые данные новый признак 'NEW', равный произведению признаков '7' и '11'. На тренировочных данных с новым признаком заново с помощью GridSearchCV (с тремя фолдами и метрикой качества - f1-score) подберите оптимальное значение C (перебирайте те же значения C, что и в предыдущих заданиях), в ответ напишите наилучшее качество алгоритма (по метрике f1-score), ответ округлите до сотых.
- При объявлении модели фиксируйте random_state = 42.

3. Теперь вы можете использовать любую модель машинного обучения для решения задачи. Также можете делать любую другую обработку признаков. Ваша задача - получить наилучшее качество по метрике F1-Score на тестовых данных.

In [ ]:
# a) Подбор оптимального значения C в логистической регрессии
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
model = LogisticRegression(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro')
grid_search.fit(X, y)
best_c = grid_search.best_params_['C']
print(f"Оптимальное значение C: {best_c}")

# b) Добавление нового признака и подбор оптимального C
train_df['NEW'] = train_df['feature_7'] * train_df['feature_11']
test_df['NEW'] = test_df['feature_7'] * test_df['feature_11']

X_train = train_df.drop('target', axis=1)
y_train = train_df['target']

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}
model = LogisticRegression(random_state=42)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='f1_macro')

In [1]:
import numpy as np

# Задаем исходные данные
X = np.array([[-1, 1], [1, -1], [1, 1], [0, 0]])
y = np.array([1, 1, 1, -1])

# Функция для расчета евклидова расстояния
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

# Функция для реализации метода k-ближайших соседей
def knn_classify(X, y, x_new, k):
    distances = [euclidean_distance(x_new, x) for x in X]
    sorted_indices = np.argsort(distances)
    
    nearest_labels = [y[i] for i in sorted_indices[:k]]
    return np.sign(np.sum(nearest_labels))

# Функция для реализации leave-one-out кросс-валидации
def leave_one_out_cv(X, y, k_values):
    accuracies = []
    for k in k_values:
        correct_predictions = 0
        for i in range(len(X)):
            X_test = X[i]
            y_test = y[i]
            X_train = np.delete(X, i, axis=0)
            y_train = np.delete(y, i)
            prediction = knn_classify(X_train, y_train, X_test, k)
            if prediction == y_test:
                correct_predictions += 1
        accuracy = correct_predictions / len(X)
        accuracies.append(accuracy)
    return accuracies

# Проводим leave-one-out кросс-валидацию для k=[1, 2, 3]
k_values = [1, 2, 3]
accuracies = leave_one_out_cv(X, y, k_values)

print("Результаты leave-one-out кросс-валидации:")
for k, acc in zip(k_values, accuracies):
    print(f"k = {k}, accuracy = {acc:.2f}")

# Находим оптимальное значение k
optimal_k = k_values[np.argmax(accuracies)]
print(f"Оптимальное значение k: {optimal_k}")


Результаты leave-one-out кросс-валидации:
k = 1, accuracy = 0.00
k = 2, accuracy = 0.00
k = 3, accuracy = 0.75
Оптимальное значение k: 3
